In [ ]:
'''Импортируем необходимые библиотеки. '''

In [92]:
import vk
import sqlite3
from time import time, sleep, strptime, strftime, mktime, gmtime

In [ ]:
'''Если модуль отсутствует: 
Jupiter: установим его через !pip install
Pycharm: check interpreter and path, see file -> settings -> python interpreter -> packages'''

In [ ]:
'''If there are some problems with time library:
Since time is a standart library, we don't have to import it to pyCharm, just use it
Or try to import it with a name: i.e. import time as tm
Check all the references to the imported functions from module time thoroughly'''

In [93]:
groups = ['grazhdanesb', 'mayaksbor', 'stv24']

In [94]:
chunks = [groups]

In [95]:
chunks

[['grazhdanesb', 'mayaksbor', 'stv24']]

In [ ]:
''' The following line below creates NEW database, be careful'''

In [108]:
my_file = open("base.db", "w+")

In [97]:
conn = sqlite3.connect(r'base.db')
c = conn.cursor()

In [ ]:
'''Get vk admin token instead of vk.com otherwise method wallget is deprecated with code error 3'''

In [98]:
token = "XXXXXXXXXXXXXXXXXXXXXWnZRWpRRA"
api = vk.API(access_token = token)

In [99]:
c.execute('''CREATE TABLE IF NOT EXISTS groups (ID INT, name text,screen_name text, is_closed int, type text,
           deactivated text,description text,country text,city text,contacts, members INT)''')

In [ ]:
''' VK API with standart methods takes no more than 3 requests per seconds'''

In [100]:
start_time = time()
for chunk in chunks:

    json_group = api.groups.getById(group_ids = ','.join(chunk), v=5.131,
                                    fields ='deactivated,description,members_count,country,city,contacts')
    sleep(0.32)

    for item in json_group:
        print(item)
        ID = item['id']
        name = item['name']
        screen_name = item['screen_name']
        is_closed = item['is_closed']
        tip = item['type']
        deactivated = item['deactivated'] if 'deactivated' in item.keys() else None
        desc = item['description'] if 'description' in item.keys() else None
        country = item['country']['title'] if 'country' in item.keys() else None
        city = item['city']['title'] if 'city' in item.keys() else None
        members = item['members_count'] if 'members_count' in item.keys() else None
        contacts = ''
        if 'contacts' in item.keys():
            for contact in item['contacts']:
                contacts += str(contact)
        else:
            contacts = None

        row = (ID, name, screen_name,is_closed,tip,deactivated,desc,country,city,contacts,members)
        conn.execute("INSERT INTO groups VALUES (?,?,?,?,?,?,?,?,?,?,?)", row)

    conn.commit()

conn.close()
s = int(time() - start_time)
print("--- %s seconds ---" % s)

{'id': 72889389, 'city': {'id': 1048255, 'title': 'Сосновый Бор'}, 'country': {'id': 1, 'title': 'Россия'}, 'description': '«Граждане города» и горожане – плечом к плечу!\n\nСосновый бор официальный сайт Ассоциации "Граждане города Сосновый Бор".\n\nОбщественное движение «Граждане города» изменяет свой статус, продолжая развитие. Наша группа не может отставать от общей тенденции, и сегодня мы заявляем о полноценном изменении ее формата!\nОценив состояние информационного пространства нашего города, мы уверены, что горожанам необходим настоящий информационно-развлекательный центр, ядром которого и станет наша группа. \nИменно здесь мы постараемся собрать всю актуальную и необходимую для горожан информацию, обсудим важнейшие городские проблемы, проведем интересные конкурсы, которые помогут горожанам лучше узнать любимый город. \nНа страницах нашей группы Вы сможете задать интересующие и насущные вопросы депутатам городского собрания представляющих наше движение и получить на них ответы. \

In [101]:
conn = sqlite3.connect(r'base.db')
c = conn.cursor()

c.execute('''CREATE TABLE IF NOT EXISTS posts
      (UID INTEGER,date date, owner_id INTEGER, post_id INTEGER, from_id INTEGER,text text,
         comments INTEGER, likes INTEGER, reposts INTEGER,views INTEGER, post_type text,attachments text,
         history text)''')

c.execute('''SELECT * FROM groups''') # нам не надо никаких условий

group_data = [-int(list(item)[0]) for item in c.fetchall()]
print(group_data)

[-72889389, -405719, -16374902]


In [102]:
time_threshold = '2022-02-24 00:00:00'
time_threshold_epoch = int(mktime(strptime(time_threshold, '%Y-%m-%d %H:%M:%S'))) #convert to UNIX timestamp

In [107]:
#FOR POST EXTRACTING
start_time = time()
UID = 0
print('Количество групп:', len(group_data))
for i in range(len(group_data)):

    print("Группа"+str(i+1)+": "+str(group_data[i]))
    print('-------------------------------------')
    try:
        json_form_num = api.wall.get(owner_id = group_data[i],offset=0,count=100,v=5.131);
        sleep(0.33)
    except vk.exceptions.VkAPIError as e:
        if e.code == 15 or e.code == 18 or e.code == 30:
            print("Ошибка 15 или Ошибка 18 или Ошибка 30")
            continue
        else:
            raise

    print("Число постов:"+str(json_form_num['count']))

    offset = 0
    
    f = round(json_form_num['count']/100+0.499)

    for j in range(0, f):
        try:
            json_form = api.wall.get(owner_id = group_data[i],offset=offset,count=100,v=5.131);
            sleep(0.33)
        except vk.exceptions.VkAPIError as e:
            if e.code == 15:
                break
            else:
                raise
        if j % 10 == 0:        
            print(round(j/f*100, 2), '% completed')
        #print(json_form)


        for post in json_form['items']:
            UID += 1

            if post['text'] is not None:
                text = post['text'].replace("\n", " ")
            else:
                text = None

            if "views" in post.keys():
                views = post['views']['count']
            else:
                views = None

            if "likes" in post.keys():
                likes = post['likes']['count']
            else:
                likes = None

            if "reposts" in post.keys():
                reposts = post['reposts']['count']
            else:
                reposts = None

            if "attachments" in post.keys():
                attachments = str(post['attachments'])
            else:
                attachments = None

            if "copy_history" in post.keys():
                history = str(post['copy_history'])
            else:
                history = None

            row = (UID,
                   strftime("%Y-%m-%d %H:%M:%S", gmtime(post['date'])),
                   group_data[i],
                   post['id'],
                   post['from_id'],
                   text,
                   post['comments']['count'],
                   likes,
                   reposts,
                   views,
                   post['post_type'],
                   attachments,
                   history
                   )

            conn.execute("INSERT INTO posts VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", row)
        offset += 100
        conn.commit()
        if json_form['items'][-1]['date'] < time_threshold_epoch: # тоже убрать коммент
            print('threshold is reached')
            break
conn.close()
mnt = int(time() - start_time)/60
print("--- %s minutes ---" % mnt)

Количество групп: 3
Группа1: -72889389
-------------------------------------
Число постов:106888
0.0 % completed
2.34 % completed
4.68 % completed
7.02 % completed
threshold is reached
Группа2: -405719
-------------------------------------
Число постов:61817
0.0 % completed
4.04 % completed
threshold is reached
Группа3: -16374902
-------------------------------------
Число постов:29479
0.0 % completed
8.47 % completed
threshold is reached
--- 3.95 minutes ---
